<img src="../figures/header.png" alt="Header" width="450"/>

# Prompt Engineering 01: Técnicas Básicas

En los últimos meses, los LLMs (Large Language Models) han demostrado ser una herramienta sumamente útil y poderosa para un sin fin de tareas. De hecho, medio año después del lanzamiento oficial de [ChatGPT](https://openai.com/blog/chatgpt), todavía estamos rascando la superficie y descubriendo nuevos usos potenciales cada semana. Pero no todo es color de rosas, estos modelos todavía no son lo suficientemente desarrollados para "leer nuestras mentes". Por esto, necesitan recibir instrucciones claras y libres de ambigüedades, para ejecutar tareas de forma exitosa. En pocas palabras, si queremos liberar el verdadero potencial de los LLMs necesitamos aprender a hablar su idioma.

**¿Qué vamos a ver en este Notebook?**

En este Notebook, vamos a recorrer los principios fundamentales del Prompt Engineering y profundizaremos en algunas de las técnicas más utilizadas para la elaboración de prompts. Estos principios y técnicas son en su mayoría agnósticos al modelo de lenguaje utilizado (ChatGPT, GPT-4, LLaMA, etc.), o al menos, válidos para la gran mayoría de modelos generativos de tipo [Text-to-Text](https://huggingface.co/tasks/text-generation#text-to-text-generation-models).

**¿Qué herramientas vamos a utilizar?**

Por simplicidad y afinidad a la implementación actual de los productos de Cognitive Solutions, para los ejemplos de prompts vamos a utilizar el modelo más reciente de OpenAI disponible a través de la API oficial (al momento de elaborar estos tutoriales dicho modelo era `gpt-3.5-turbo`). Además, para consumir la API de OpenAI vamos a utilizar el [sdk](https://pypi.org/project/openai/) oficial, que se encuentra disponible en pypi. Para el resto, funciones y lógica adicional para el procesamiento de las respuestas del modelo, vamos a utilizar puro Python y alguna librería sencilla para algún problema específico pero que no merece la pena mencionar todavía.

Por último, es necesario contar con una API Key válida con créditos disponibles para realizar llamadas a la API de Open AI. Para más información sobre cómo generar y administrar las API Key puedes revisar la sección [OpenAI API Key setup](#openai-api-setup).

## Índice
<a class="anchor" id="index"></a>

1. [Setups](#setups)
    1. [OpenAI API Key setup](#openai-api-setup)
    2. [Import & Utils](#imports)
2. [¿Qué es Prompt Engineering?](#what-is-prompt-engineering)
3. [Abstracciones de la API de OpenAI](#openai-playground)
4. [Principios de Prompt Engineering](#principles-of-prompt)
    1. [Principio 1: Ser claro y específico](#be-clear-and-specific)
    2. [Principio 2: Darle tiempo para pensar](#time-to-think)
    3. [Principio 3: Balancear especificidad con creatividad](#allow-creativity)
5. [Consejos Útiles](#tips-and-tricks)
6. [References](#references)

## Setups
<a class="anchor" id="setups"></a>

### 1. OpenAI API Key setup
<a class="anchor" id="openai-api-setup"></a>

ToDo

* Mostrar con capturas de pantalla o similar como configurar una API Key de OpenAI desde el panel de control para utilizar aquí. También hacer hincaṕie en que vamos a proveerlas nosotros a los efectos del bootcamp.
* Mostrar como poner la API key en .env para que no quede harcodeada en el notebook

Para simplificar el manejo de las API Keys y evitar que queden "harcodeadas" en este notebook, utilizamos la librería [python-dotenv](https://pypi.org/project/python-dotenv/) que permite facilmente cargar variables de entorno definidas en un archivo local. En pocas palabras, lo único que necesitamos asegurarnos es que exista un archivo de nombre `.env` en la raiź de este proyecto, con el siguiente contenido:


```yaml
OPENAI_API_KEY=[A_VALID_OPENAI_API_KEY]
```


Como parte de la sección **Import&Utils** vamos a cargar esta variable de entorno utilizando los métodos `load_dotenv` y `find_dotenv` de esta libreria y finalmente setear las clases correspondientes del sdk de OpenAI para que utlicen la API Key cargada en memoria. 

### 2. Imports & Utils
<a class="anchor" id="impots"></a>

In [1]:
import openai
import os
import pandas as pd

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')  # Be sure you have a .env file defined with your OPENAI_API_KEY 

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=1.0, top_p=1.0):
    """
    
    """
    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        top_p=top_p
    )
    return response.choices[0].message["content"]

def save(path: str, text: str):
    """
    Helper function to save prompts and model responses as TXT files
    """
    
    with open(f"{path}.txt", "w") as text_file:
        text_file.write(text)

In [3]:
"""
Styles:

This cell is to provide CSS styles for the model badge that indicates which model was used for some 
specific prompt.
"""

from IPython.core.display import HTML
HTML("""
<style>

.label {
  color: white;
  padding: 5px;
}

.success {background-color: #04AA6D;} /* Green */
.info {background-color: #2196F3;} /* Blue */

</style>
""")

In [4]:
# data = pd.read_csv("../data/faqs.csv")

In [5]:
# data

## ¿Qué es Prompt Engineering? 
<a class="anchor" id="what-is-prompt-engineering"></a>

🔝 [Ir al Índice](#index)

**Intuitivamente**, un prompt no es nada más que un texto que se introduce en un Modelo de Lenguaje (LM), y “Prompt Engineering” (o Ingeniería de Prompts) es el arte de diseñar dicho texto para obtener el resultado deseado. Algunas personas bromean con que los Prompts son como hechizos: un conjunto de palabras, que siguen una serie de reglas complejas y extravagantes, que logran lo imposible.

Más allá de la bella metáfora, por supuesto que no hay nada magico detrás del Prompt Engineering. Los Modelos de Lenguaje son simplemente modelos estadísticos para texto, y Prompt Engineering es simplemente un conjunto de técnicas para diseñar un input text que contenga los tokens precisos (por ejemplo instrucciones o contexto) para orientar al modelo en la dirección indicada y generar un buen resultado (más tokens o incluso una imagen). 


**Formalmente**, Prompt Engineering es el proceso integtral de diseñar y optimizar prompts específicos y libres de ambiguedades, para Large Language Models (LLMs), que permitan asegurar la generación de respuestas relevantes, precisas. Implica confecionar prompts que sean claros y concisos, y a su vez que incluyan suficiente información de contexto para que el modelo comprenda el input de usuario y genere una respuesta apropiada. También implica una comprensión profunda de las capacidades y limitaciones del modelo, y suele requerir de varias iteraciones y pruebas para alcanzar el resultado deseado.

En esencia, el proceso iterativo de desarrollar prompts es muy similar al de escribir software. Si no se logra el resultado deseado, refinar el prompt y repetir hasta que se obtenga el resultado deseado.


Diseñar Prompts efectivos es esencial para crear agentes conversacionales que puedan comprender y responder a consultas en lenguaje natural, de una manera que sea útil y atractiva para los usuarios. Además es de vital importancia para prevenir situaciones en las que el modelo pueda generar respuestas sin sentido, inapropiadas o perjudiciales.

## Abstracciones de la API de OpenAI
<a class="anchor" id="openai-playground"></a>

🔝 [Ir al Índice](#index)

Antes de pasar a los principios fundamentales de Prompt Engineering, es importante comprender las abstracciones básicas de la API de OpenAI. Esto nos permitirá utilizar de forma más efectiva la misma y obtener mejores resultados.


De todos los servicios disponibles en la API de OpenAI, vamos a utilizar el servicio [Chat](https://platform.openai.com/docs/api-reference/chat/create) el cual permite interactuar con el modelo `gpt-3.5-turbo`, similar al modelo que alimenta la interfaz de ChatGPT.


**Request body**

Veamos los parámetros más importantes y relevantes para este tutorial, presentes en la estructura de un request a este servicio:

* `model`: Identificador del modelo a utilizar (ver lista de modelos disponibles [aquí](https://platform.openai.com/docs/models/model-endpoint-compatibility))
* `messages`: Lista de mensajes contenidos en la conversación con el servicio.
* `temperature`: Temperatura de muestreo para la selección de tokens (0 a 2)
* `top_p`: Alternativa de muestreo denominada "nucleus sampling" (0 a 1)
* `max_tokens`: Máximo de tokens a utilizar en la respuesta generada
* `functions`: Permite definir una lista de funciones a ser llamadas por el modelo para la generaciónd de la respuesta, esto permite conectar el modelo con Plugins.

A modo de ejemplo, el siguiente snippet de código Python ejemplifica como replicar esta estructura para interactuar con el sdk de openai.

In [4]:
prompt = "Contame un chiste sobre manzanas"

messages = [{"role": "user", "content": prompt}]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0, # this is the degree of randomness of the model's output
    top_p=1.0
)

print(response.choices[0].message["content"])

Claro, aquí tienes uno:

¿Qué le dijo una manzana verde a una manzana roja?

¡Eres tan "apple" tada!


La mejor forma de conocer en profundidad los parámetros disponibles de OpenAI y sus efectos en la generación de respuestas, es jugar con los mismos y probar diferentes valores. Sientete libre de modificar los valores en este notebook para probar variantes. Otra alternativa es hacerlo a través del API [playground](https://platform.openai.com/playground) de OpenAI que permite hacer lo mismo mediante una interfaz web.

**Messages**
En la estructura del request, `messages` es una lista de mensajes, en lugar de un único mensaje. Esto es una decisión de diseño deliverada que permite enviar como parte del request, todos los mensajes que componen la conversación que estamos manteniendo con el modelo y de esta forma implementar una suerte de "memoria". En pocas palabras, cuando mantenemos una conversación con un LLM como ChatGPT, no es que el modelo "se acuerde" de nuestra conversación si no que por el contrario, en cada nueva invocación se provee como parte del contexto, la conversación mantenida anteriormente.

**Message Role**

Si tenemos la posibilidad de proveer toda la conversación como parte del contexto en la invocación al modelo, entonces como hace el modelo para distinguir entre los mensajes del usuario y los generados por el propio modelo, o incluso de las instrucciones del prompt? Para esto los mensajes tienen un campo `role` que indica que tipo de mensaje es.

* `system`: Mensaje que contiene las instrucciones para el modelo, tipicamente llamado prompt.
* `assistant`: Mensaje producido por el sistema en respuesta al prompt o mensje de ususario
* `user`: Mensaje de usuario, tipicamente generado en respuesta a la salida del modelo para continuar una conversación.

Más información sobre los tipos de mensaje en OpenAI y el resto de los parámetros de la API [aquí](https://platform.openai.com/docs/api-reference/chat). Para los que estan pensando en Langchain y como se implementa esto, son contemplados en los [chat-models](https://blog.langchain.dev/chat-models/).

## Principios de Prompt Engineering
<a class="anchor" id="principles-of-prompt"></a>

🔝 [Ir al Índice](#index)

Intuitivamente existen tres principios detrás de la escritura de prompts efectivos que te guiarán en la generación de los resultados que estas buscando.

<img src="../figures/2.png" alt="Principles of Prompt Engineering" />

## Primer Principio: Escribir instrucciones claras y específicas (Be clear and specific)

<a class="anchor" id="be-clear-and-specific"></a>

🔝 [Ir al Índice](#index)

La forma más eficaz de obtener el resultado deseado de un LLM, es proveer instrucciones lo más claras y específicas que sea posible. Esto reduce las chances de obtener respuestas incorrectas o poco relevantes para nuestro contexto. Sin embargo, nunca confundir instrucciones claras con instrucciones cortas. En algunos escenarios, prompts más largos de hecho proveen mayor claridad e información de contexto que le permiten al modelo generar respuestas correctas y detalladas.

A continuación vemos algunas tácticas que puedes seguir para escribir prompts claros y espcíficos.

### 1. Usar delimitadores

Utilizar delimitadores en el prompt para separar las instrucciones (instrucciones al sistema) del input del usuario. Cualquier caracter es váido como separador (por ejemplo simbolos de puntuación), conviene elegir un símbolo poco frecuente para reducir la probabilidad de aparición como parte del input de usuario.

* Comillas triples `“““`
* Triple acento (back-ticks) \`\`\`
* Guión triple (dashes) `---`
* Numeral Triple `###`
* Angle brackets `< >`
* Tags XML `<tag> </tag>`

Tener en cuenta que caracteres delimitadores largos o muy verbose, generan prompts más largos (en cantidad de tokens), lo cual conduce a un mayor consumo de APIs y a la ves podría ser un problema con prompts que se acerquen al límite máximo de tamaño (4095 para GPT-4, más info [aquí](https://community.openai.com/t/chatgpt-4-context-lengths/114919/3)).

Se puede utilizar el [Tokenizer](https://platform.openai.com/tokenizer) de OpenAI para verificar cuantos tokens ocupa un caracter delimitador en particular. Por ejemplo:

* Comillas triples `“““` → 1 Token
* Triple acento (back-ticks)  \`\`\` → 2 Tokens
* Guión triple (dashes) `---` → 1 Token
* Numeral Triple `###` → 1 Token
* Angle brackets `< >` → 2 Tokens
* Tags XML `<tag></tag>` → 5 Tokens

Por otro lado, utilizar delimitadores es útil para evitar inyecciones de prompt, al separar explícitamente las instrucciones al modelo del resto del prompt.

Veamos esta táctica en la práctica con un ejemplo real. Imagine que estamos trabajando para mejorar un sitio web de comercio electrónico que brinda mejores perspectivas al equipo de ventas en función de las reseñas de los clientes. Por ejemplo, si los compradores mencionan con frecuencia que la talla de un producto es demasiado grande o demasiado pequeña, esto podría indicar que la tabla de tallas real no es precisa.

Sin embargo, analizar manualmente las opiniones de los clientes es una tarea tediosa y que requiere mucho tiempo. Por este motivo, utilizaremos GPT-4 para examinar estas revisiones y proporcionar un resumen completo.

Los siguientes ejemplos son reseñas reales que los compradores en línea han hecho de una campera para la nieve [Columbia Men's Republic Parka (Obi-Wan Kenobi™)](https://www.columbia.com/p/mens-republic-parka-obi-wan-kenobi-2049261_OWK.html). ¡Qué por cierto se ve tan genial! ¡Si no costara 500 dólares, también lo compraría!

In [5]:
reviews = """
1 Star
This is an over engineered waste of money

Nobody is truly going to be wearing this unless your in the frozen tundra…. Zero mobility way to large in length… Absolute waste…

---
5 Stars
Incredibly warm!

Incredibly warm! You won’t go wrong with this jacket!

---
5 Stars
So warm!

I am a 5”4 woman and got this men’s jacket as a gift. I moved from sunny Los Ángeles to Detroit Michigan during the storm and this jacket saved me. I get cold easily and I was so warm while walking in the snow that I had to unzip a bit because I was getting too warm. Love that this jacket has so many pockets, I don’t even need to bring a purse cuz this jacket holds it all. The details they put into this jacket is amazing.

---
1 Star
Waste of money great look but just not designed to be used and it’s shown by the resale value

Absolutely way too bulky and does not fit as described… This jacket is over engineered meaning unless your going to the polar ice caps you won’t need it. Was hoping for a solid winter jacket and this is definitely not it. Zero mobility in this with how bulky it is… And I have other jackets by Columbia… Also Ordered an XL just as my other jackets but this one is basically a XXL the jacket is also extremely long for no reason. Definitely will be returning this as the package also came opened and the jacket was dirty.

---
2 Stars
Overpriced, and runs big

I love Star Wars and really loved this jacket in concept. It took forever to ship and when it did finally show up the package was ripped open and the jacket was dirty and damaged. Customer service was not helpful and I kept getting kicked from chat. They wanted me to return it and rebuy while doing so meanwhile I have spent $1000 and have no jacket for most of winter. Not worth the money or the hassle. Guess I am still a north face guy.

---
5 Stars
I love it!!

A very nice parka!! Comfy and warm and it looks great! Very glad I purchased it.

---
5 Stars
Awesome

Warmest jacket I've ever had. Lots of pockets, looks excellent

---
2 Stars
Very nice looking jacket

The jacket looks really nice and is warm . The sizing chart is completely off. I would normally be a large to extra large because of wide shoulders and back (I matched the large I'm the size chart). I can almost fit 2 of me into this jacket . Definitely go a size down , maybe even 2 . If you an athletic build .
"""

Comencemos con un prompt sencillo, indicando al modelo que analice las opiniones de clientes y genere un resumen de la percepción general. Con el espíritu de hacer los ejemplos más limpios y fáciles de leer, separaremos las instrucciones del sistema del resto de la entrada. Para hacer esto, usaremos [string literals](https://docs.python.org/3/reference/lexical_analysis.html?ref=hackernoon.com#formatted-string-literals) de Python. Además, vamos a proveer el prompt en inglés, ya que los modelos suelen ser un poco más robustos en este idioma. No obstante, alentamos al lector a que pruebe de cambiar las instrucciones a español para apreciar por si mismo si hay cambios importantes en los resultados.

**Prompt 💬**

In [6]:
prompt = f"""
You will be provided with a list of product reviews delimited by <>.
Analyze each reviews and provide an overview of customers satisfaction, listing 
positive and negative aspects reported by customers. 

<{reviews}>
"""

El prompt inicial es extremadamente simple y específico, utilizando <> para separar las instrucciones del resto de la entrada (puede usar comillas invertidas, comillas triples o cualquier otro carácter). A pesar de su sencillez, el modelo capta la idea y genera una respuesta correcta, identificando y enumerando los aspectos positivos y negativos mencionados por los compradores en sus valoraciones. En particular, el modelo identifica adecuadamente los aspectos destacados por los compradores como el amplio espacio de almacenamiento, la sensación de comodidad y calidez, entre otros. Al igual que los aspectos positivos, reconoce algunas quejas de los clientes, como problemas de tamaño y problemas con el envío del producto.

Notar que el modelo cita algunos aspectos que no están estrictamente incluidos en las opiniones de los clientes. Por ejemplo: amplio espacio de almacenamiento. Los clientes mencionaron varios bolsillos pero nunca dijeron claramente "amplios" (esto es una suposición del modelo). Además, ninguno se queja específicamente del precio de la campera. A este tipo de discrepancias e inconsistencias con los datos en el contexto, se le suelen denominar [Alucinaciones](https://en.wikipedia.org/wiki/Hallucination_(artificial_intelligence) (esta es una muy leve), y es un [problema común](https://spectrum.ieee.org/ai-hallucination) de los LLMs. Lo bueno es que existen tácticas que ayudan a reducir estas alucinaciones. 


**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
"""
Overview of customer satisfaction:

Positive aspects:

1. Incredibly warm and suitable for cold weather conditions.
2. Multiple pockets providing ample storage space.
3. Attractive design and aesthetics.
4. Comfortable and good fit for some customers.

Negative aspects:

1. Over-engineered, bulky, and may not be suitable for everyday wear.
2. Sizing issues, with some customers reporting that the jacket runs large.
3. Poor customer service experiences, including shipping issues and unhelpful responses.
4. Expensive and potentially not worth the price for some customers.
"""
```

Ahora te toca a tí, prueba con la siguiente celda de código!

In [7]:
response = get_completion(prompt)
print(response)

Overview of customers satisfaction:
- Overall, the majority of customers seem to be satisfied with the product, as there are multiple 5-star reviews praising the jacket for being warm, comfortable, and having a nice design.
- However, there are also a few negative reviews, with customers expressing dissatisfaction with the jacket's fit, mobility, and customer service. Some customers also felt that the jacket was overpriced.

Positive aspects reported by customers:
- The jacket is incredibly warm and can keep customers comfortable in cold weather.
- It has multiple pockets, which is convenient and eliminates the need for a purse.
- The details and design of the jacket are praised by customers.
- Some customers describe the jacket as comfy, great-looking, and excellent.

Negative aspects reported by customers:
- The jacket is considered too bulky and long, leading to mobility issues.
- Customer service was reported as unhelpful and caused inconvenience to some customers.
- The sizing cha

### 2. Usar respuestas estructuradas

Es muy útil instruirle al modelo que genere su respuesta siguiendo un formato estructurada como pueden ser `JSON`, `XML` o cualquier otro formato dentro de los limites de comprensión del modelo. Esta táctica reduce la complejidad de la respuesta, elimina jerga innecesaria y hace mucho más simple el procesamiento de la respuesta de forma automática utilizando algún lenguaje de programación como por ejemplo Python. Así que volvamos a nuestro prompt inicial y actualicemos las instrucciones ahora pidiéndole al modelo que utilice un formato JSON en la salida, con dos listas: aspectos positivos y negativos.

In [8]:
prompt = f"""
You will be provided with a list of product reviews delimited by <>.
Analyze each reviews and provide an overview of customers satisfaction, listing 
positive and negative aspects reported by customers. 

Provide them in JSON format with the following keys: 
positive_aspects, negative_aspects.

<{reviews}>
"""

La nueva respuesta es más específica, con menos espacio para la creatividad y fácil de procesar mediante código. Esto introduce otro aspecto crítico del prompt engineering: equilibrar la especificidad con la creatividad. Si bien ser específico es crucial, también es importante dejar espacio para que el modelo demuestre sus capacidades creativas. Lograr el equilibrio adecuado entre especificidad y creatividad puede conducir a resultados más atractivos y útiles.

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
{
  "positive_aspects": [
    "Incredibly warm",
    "Many pockets",
    "High-quality details",
    "Comfortable",
    "Stylish",
    "Excellent appearance"
  ],
  "negative_aspects": [
    "Over-engineered",
    "Bulky",
    "Limited mobility",
    "Sizing issues",
    "Expensive",
    "Poor customer service",
    "Shipping issues",
    "Dirty and damaged upon arrival"
  ]
}
```

In [9]:
# Now is your turn!
response = get_completion(prompt)
print(response)

{
  "positive_aspects": [
    "Incredibly warm",
    "Lots of pockets",
    "Looks great",
    "Comfy",
    "Warm",
    "Nice looking jacket"
  ],
  "negative_aspects": [
    "Over engineered",
    "Zero mobility",
    "Too large in length",
    "Waste of money",
    "Not designed to be used",
    "Bulky",
    "Dirty and damaged",
    "Overpriced",
    "Customer service not helpful",
    "Sizing chart is off"
  ]
}


### 3. Chequear casos borde

Cuando diseñamos prompts basados en textos ingresados por un usuario, es una buena práctica asegurarse de que estos textos de entrada cumplan con ciertos criterios mínimos necesarios para completar las instrucciones. Si no, devolver un mensaje de error conocido. Al hacerlo, podemos garantizar que el modelo responda de manera controlada a cualquier entrada que exceda el alcance para el cual se diseñaron inicialmente el prompt y la aplicación construida sobre el LLM.

Por ejemplo, en la siguiente review de un comprador, se promociona una campera de otra marca. Para evitar que introduzca ruido en nuestro análisis, vamos a modificar el prompt para ignorar automáticamente este tipo de reviews.


(Review adicional)

In [10]:
additional_review = """
---
1 Star
Not a real Jacket

Looking for a real jacket, suitable for any weather condition and ready for the highest demands? Check out our McMurdo Men’s McMurdo Parka, the best value for money on the market.
"""

reviews += additional_review

Primero, vamos a iterar sobre el prompt que tenemos, incorporando la capacidad de distinguir entre reviews válidas e inválidas. La forma de hacerlo es decirle al modelo como distinguirlas (reglas) o explícitamente mostrarle ejemplos de review válidas e inválidas. En el siguiente prompt, le indicamos al modelo que ignore las revisiones no relacionadas. Te aliento a que hagas tu propia prueba con el modelo.

(Prompt intermedio)

In [11]:
prompt = f"""
You will be provided with a product review about a Columbia Jacket, delimited by 
<>.

If the text is related to the product, provide an overview of customer satisfaction, listing positive and negative aspects reported by the customer. 

If the text is not related to the product, then simply write \"The provided review is not related to the product or is a Spam.\"

{reviews}

"""

In [12]:
# Now is your turn!
response = get_completion(prompt)
print(response)

The provided review is not related to the product or is a Spam.


Ahora que sabemos cómo validar una condición sobre la entrada, combinemos esto con el prompt original para agregar un contador de reviews ignoradas como parte de la respuesta `JSON`.

**Prompt 💬**

In [14]:
prompt = f"""
You will be provided with a list of product reviews about a Columbia Jacket delimited by <>.
Analyze each reviews and provide an overview of customers satisfaction, listing 
positive and negative aspects reported by customers. 

It may happen that some of the reviews are not strictly related to the product, in that case ignore the review, 
update a counter of ignored reviews that you will display with the rest of the output and continue with the next review. 

Provide them in JSON format with the following keys: 
positive_aspects, negative_aspects, ignored.

<{reviews}>
"""

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
{
  "positive_aspects": [
    "Incredibly warm",
    "Many pockets",
    "Great details",
    "Comfy",
    "Looks great",
    "Excellent appearance"
  ],
  "negative_aspects": [
    "Over engineered",
    "Lack of mobility",
    "Too large in length",
    "Bulky",
    "Sizing issues",
    "Poor customer service",
    "Delayed shipping",
    "Package received damaged and dirty"
  ],
  "ignored": 1
}
```

Mucho mejor... Tener en cuenta que este prompt se apoya en el conocimiento actual del modelo para distinguir una reseña sobre una campera Columbia (válida) de cualquier otro texto (no válida). Para este ejemplo es suficiente, pero existen técnicas más sólidas para enseñarle al modelo a distinguir entre entradas válidas e inválidas. Una de las técnicas más robustas es la conocida como few-shot prompting cubierta en la siguiente sección.

In [15]:
# Now is your turn!
response = get_completion(prompt)
print(response)

{
  "positive_aspects": [
    "Very warm",
    "Lots of pockets",
    "Looks great",
    "Comfy and warm",
    "Excellent parka"
  ],
  "negative_aspects": [
    "Over-engineered",
    "Zero mobility",
    "Bulky",
    "Does not fit as described",
    "Long length",
    "Overpriced",
    "Runs big",
    "Package arrived opened and dirty",
    "Customer service not helpful",
    "Sizing chart is off",
    "Not suitable for all weather conditions"
  ],
  "ignored": 2
}


### 4. Few-shot prompting

En algunos escenarios, proporcionar ejemplos de ejecuciones exitosas de la tarea y luego pedirle al modelo que la ejecute, es una estrategia inteligente para obtener resultados más precisos y predecibles. Para ilustrar esta táctica, pensemos momentáneamente en una funcionalidad relacionada en el sitio web de comercio electrónico. Imaginemos que ahora queremos generar automáticamente un mensaje de respuesta para cada opinión de cliente.

Para ello, le vamos a decir al modelo cómo responder a algunos ejemplos típicos de reviews de productos.

**Prompt 💬**

In [16]:
prompt = f"""
You will be provided with a list of product reviews delimited by <>.
Analyze each review and provide a personalized answer for each one, based on the content of the review.

Here is a few examples:

---
Review: I am very happy with this jacket, it is worth every penny
Response: Thank you for your review, we are very happy that you are satisfied with our products.

---
Review: I normally wear Columbia size L jackets, this in L is like an XL. I reordered the M… unfortunately the blue was not longer available =(
Response: Thank you for your review! We are continuously working to bring you a better shopping experience and innovative products that will surprise you.

---
Review: Nobody is truly going to be wearing this unless your in the frozen tundra…. Zero mobility way to large in length… Absolute waste…
Response: Thank you for your review, the feedback from our users represents an invaluable asset for us that allows us to work to improve each product and bring you a better shopping experience. 

Provide them in JSON format with the following keys: review, response. Where review is the first 10 words of the customer 
review to easy identify it and response is the generated response.

<{reviews}>
"""

Nótese que el modelo genera diferentes respuestas, según el mensaje del usuario, con un tono similar a los ejemplos proporcionados. Puedes ampliar la lista de ejemplos proporcionados en el prompt, para mostrarle al modelo cómo actuar en diferentes circunstancias si quieres. Además, tenga en cuenta que el modelo no sigue estrictamente nuestras especificaciones. El resumen de la revisión tiene una extensión de cinco palabras en lugar de diez. Esto puede indicar que todavía tenemos que trabajar en nuestro prompt para que sea más claro y específico.

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
{
  "1": {
    "review": "This is an over engineered waste",
    "response": "Thank you for your review. We're sorry that the product didn't meet your expectations. We value your feedback and will consider it for future product improvements."
  },
  "2": {
    "review": "Nobody is truly going to",
    "response": "Thank you for your review. We apologize if the product didn't meet your needs. We appreciate your feedback and will use it to improve our products and ensure they cater to a wider range of customers."
  },
  "3": {
    "review": "Incredibly warm! You won’t",
    "response": "Thank you for your review! We're glad to hear that you're happy with the warmth of the jacket. We hope it continues to serve you well in the colder months!"
  },
  "4": {
    "review": "I am a 5”4 woman",
    "response": "Thank you for your review! It's great to hear that the jacket has been keeping you warm and you're enjoying the many pockets. We appreciate your feedback and hope the jacket continues to serve you well!"
  },
  "5": {
    "review": "Absolutely way too bulky and",
    "response": "Thank you for your review. We apologize for the inconvenience you've experienced with the sizing and bulkiness of the jacket. Your feedback is important to us, and we'll take it into consideration for future product improvements."
  },
  "6": {
    "review": "I love Star Wars and",
    "response": "Thank you for your review. We're sorry to hear about the issues you experienced with shipping and customer service. Your feedback is valuable to us, and we'll work on improving our processes to provide a better experience in the future."
  },
  "7": {
    "review": "A very nice parka!! Comfy",
    "response": "Thank you for your review! We're happy to hear that you're enjoying the comfort and warmth of the parka. We hope it continues to serve you well in the colder months!"
  },
  "8": {
    "review": "Warmest jacket I've ever had.",
    "response": "Thank you for your review! It's great to hear that this is the warmest jacket you've ever had. We hope you continue to enjoy its warmth and style during the colder months!"
  },
  "9": {
    "review": "The jacket looks really nice",
    "response": "Thank you for your review. We apologize for the issues you experienced with sizing. We appreciate your feedback and will use it to improve our sizing charts to better accommodate customers with different body types."
  }
}
```

In [17]:
# Now is your turn!
response = get_completion(prompt)
print(response)

[
  {
    "review": "This is an",
    "response": "Thank you for your review, the feedback from our users represents an invaluable asset for us that allows us to work to improve each product and bring you a better shopping experience."
  },
  {
    "review": "Incredibly warm",
    "response": "Thank you for your review! We are very happy that you are satisfied with our products."
  },
  {
    "review": "So warm! I",
    "response": "Thank you for your review, we are very happy that you are satisfied with our products."
  },
  {
    "review": "Waste of money",
    "response": "Thank you for your review, the feedback from our users represents an invaluable asset for us that allows us to work to improve each product and bring you a better shopping experience."
  },
  {
    "review": "Overpriced, and runs",
    "response": "Thank you for your review, we are continuously working to bring you a better shopping experience and innovative products that will surprise you."
  },
  {
    "review":

## Segundo Principio: Darle al modelo tiempo para pensar (Instruct to think)
<a class="anchor" id="time-to-think"></a>

🔝 [Ir al Índice](#index)

Si se le proporciona al modelo un problema que es demasiado complejo, es posible que el modelo necesite más tiempo para pensar en una respuesta adecuada. Esto significa gastar más tiempo computacional para generar la respuesta y también usar más tokens tanto en el contexto como en la respuesta. Puedes pensar en esto como pedirle al modelo que organice su proceso de razonamiento de una manera particular, por ejemplo, siguiendo una serie de pasos o realizando ciertas tareas antes de generar una respuesta final. Pero sobre todo, use más tokens en la respuesta y procesarla durante más ciclos de cómputo.

Aquí hay algunas tácticas que se pueden usar para indicarle al modelo que piense más antes de generar una respuesta.


### 1. Indicar los pasos para completar la tarea

La primera táctica, consiste en proporcionar al modelo una lista completa de pasos para resolver la tarea, cuanto más específicos sean los pasos, mejor. Volvamos a las reviews de nuestros compradores y reescribamos el prompt all modelo para hacer ambas cosas: obtener información de los clientes y responder a las reviews.

**Prompt 💬**

In [18]:
prompt = f"""
Your task is to perform the following actions: 
1 - Analyze the following product reviews delimited by 
  <> and summary each review with 1 sentence.
2 - Categorize the opinion according to the following categories: super happy, happy, neutral, upset
3 - Provide an overview of the customer satisfaction, listing positive and negative aspects mentioned in the review
4 - Provide a personalized answer for each one, based on the content of the review.
5 - Output a json list object that contains the 
  following keys for each review: summary, category, insights, answer.

Text: <{reviews}>
"""

Para este ejemplo no utilicé todas las reviews que tenemos, para reducir la longitud de la respuesta, ya que el límite de la longitud de la respuesta dejaba algunas revisiones sin analizar en la salida del modelo (hay formas más prácticas de resolver este problema pero no se encuentran dentro del alcance de este tutorial). A pesar de eso, puede verificar que el modelo genera respuestas a las reviews más concisas y las comprende mejor extrayendo información clave.

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
[
  {
    "summary": "Reviewer thinks the jacket is an over-engineered waste of money with zero mobility.",
    "category": "upset",
    "insights": {
      "positive": [],
      "negative": [
        "over engineered",
        "zero mobility",
        "large in length"
      ]
    },
    "answer": "We're sorry to hear that you found the jacket to be over-engineered and uncomfortable. We appreciate your feedback and will take it into consideration for future improvements."
  },
  {
    "summary": "Reviewer finds the jacket incredibly warm.",
    "category": "super happy",
    "insights": {
      "positive": [
        "incredibly warm"
      ],
      "negative": []
    },
    "answer": "We're thrilled to know that you're enjoying the warmth of the jacket! Thanks for sharing your positive experience."
  },
  {
    "summary": "Reviewer loves the warmth, pockets, and details of the jacket.",
    "category": "super happy",
    "insights": {
      "positive": [
        "warm",
        "many pockets",
        "amazing details"
      ],
      "negative": []
    },
    "answer": "We're glad that you love the jacket's warmth, pockets, and attention to detail. It's great to know that it has served you well during the cold weather in Detroit!"
  },
  {
    "summary": "Reviewer complains about the jacket being too bulky, not fitting as described, and receiving a dirty product.",
    "category": "upset",
    "insights": {
      "positive": [],
      "negative": [
        "too bulky",
        "does not fit as described",
        "dirty"
      ]
    },
    "answer": "We apologize for the inconvenience caused by the sizing and cleanliness issues. We appreciate your feedback and will work on improving these aspects. Please feel free to return the jacket for a refund or exchange."
  },
  {
    "summary": "Reviewer loves the parka, saying it's comfy, warm, and looks great.",
    "category": "super happy",
    "insights": {
      "positive": [
        "comfy",
        "warm",
        "looks great"
      ],
      "negative": []
    },
    "answer": "We're happy to hear that you love the parka's comfort, warmth, and style. Thank you for your positive review!"
  },
  {
    "summary": "Reviewer thinks the jacket is the warmest they've ever had, with lots of pockets and an excellent look.",
    "category": "super happy",
    "insights": {
      "positive": [
        "warmest jacket",
        "lots of pockets",
        "looks excellent"
      ],
      "negative": []
    },
    "answer": "We're delighted that you find our jacket to be the warmest you've ever had, and that you appreciate the pockets and style. Thank you for sharing your experience!"
  },
  {
    "summary": "Reviewer likes the look and warmth of the jacket but finds the sizing chart off.",
    "category": "neutral",
    "insights": {
      "positive": [
        "nice looking",
        "warm"
      ],
      "negative": [
        "sizing chart off"
      ]
    },
    "answer": "We appreciate your feedback on the jacket's look and warmth. We apologize for the sizing issue and will work to improve the accuracy of our sizing chart. Please consider exchanging for a better fit if needed."
  }
]
```

In [19]:
# Now is your turn!
response = get_completion(prompt)
print(response)

1 - Summary of reviews:
- 1 Star: Over engineered and not practical for everyday use.
- 5 Stars: Incredibly warm and a great jacket choice.
- 5 Stars: Saved the reviewer from the cold weather and has many pockets.
- 1 Star: Bulky, over engineered, and doesn't fit properly with zero mobility.
- 2 Stars: Overpriced, damaged package, and poor customer service.
- 5 Stars: Comfy, warm, and looks great.
- 5 Stars: Warmest jacket with lots of pockets, looks excellent.
- 2 Stars: Nice looking jacket but sizing is off.
- 1 Star: Not a real jacket, misleading advertisement.

2 - Categorization of opinions:
- 1 Star reviews: Upset
- 5 Stars reviews: Super happy
- 2 Stars reviews: Upset
- 3 Stars review: Neutral

3 - Customer satisfaction overview:
Positive aspects: The jacket is incredibly warm, has many pockets, looks great, and is comfortable.
Negative aspects: The jacket is over engineered, bulky, does not fit properly, has zero mobility, and there were issues with the package and customer ser

### 2. Indicarle al modelo que piense su propia solución, antes de apresuar una conclusión

La segunda táctica es similar a la anterior, pero además le pide al modelo que primero llegue a una solución por sí mismo antes de arrojar de forma prematura una conclusión. Esto es similar a, en lugar de indicarle al modelo que diga si algo es correcto o incorrecto según el contexto, indicarle al modelo que primero trabaje en su propia solución según el contexto y luego la compare con la información provista como parte de la entrada para arrojar la solución final.

Por ejemplo, si la tarea es validar las respuestas de estudiantes para un problema matemático simple como `2 + x = 4` y luego `x=?`, sería indicarle al modelo que primero encuentre una solución por sí mismo a este problema y luego la compare con la solución del estudiante.

Para ilustrar el poder de esta táctica, exploramos otra característica del sitio web de comercio electrónico: el procesamiento de devoluciones y reembolsos de compras. La idea es darle al modelo una lista de las demandas por reembolsos, procesar y evaluar si efectivamente aplica un reembolso, de acuerdo a las políticas de la empresa. Aquí hay seis ejemplos de solicitudes de reembolso por varias razones.

In [20]:
refunds = """
---
User: Jack
Text: The description of the product stated that the socks were made primarily of cotton with a bit of spandex. The product I received indicated that the socks were made of primarily of polyester and a tiny bit of spandex. I wanted cotton socks so I returned the product. Several attempts to contact the supplier went unanswered and I still have not received a refund on my credit card. I need a solution and want a refund.

---
User: Tom
Text: When I received my purchase, the package was open and significantly damaged. After unpacking it, I found that the shirt was dirty and had several scratches. I want to return and have a new unit sent to me.

--- 
User: Bob
Text: I ordered an XL just as my other Columbia jackets but this one is basically a XXL. The jacket is also extremely long for no reason. I want to return it.

--- 
User: Shara
Text: The jacket looks really nice and is warm but the sizing chart is completely off. I would normally be a large to extra large because of wide shoulders and back (I matched the large I'm the size chart). However, after using it a week I can say I need to go a size down or maybe even 2. Can I change it for a medium size?

---
User: Sasha
Text: This jacket is incredibly warm and solidly built. However, they charged my credit card twice, so I am writing to get a refund of the second charge. Thanks in advance.

---

User: Chris
Text: I bought this backpack for my children because the capacity and solid construction look. However after 2 weeks of school is completely demolished. Poor quality materials and broken zips. I want a refund.

---
"""

Para aquellos que no han tenido la alegría de recibir un producto dañado, las condiciones de devolución y reembolso no son especialmente fáciles de leer, con muchas reglas para casos específicos y mucha letra chica. Por ejemplo, imaginemos que la política de devolución y reembolso de nuestro sitio de comercio electrónico se reduce a las siguientes cuatro reglas (en un escenario real, serían mucho más que cuatro reglas).

1. El cliente recibió el producto dañado o en malas condiciones.
2. El cliente recibió el talle equivocado o un producto diferente. El producto debe estar todavía en el paquete original y sin uso.
3. El cliente no está satisfecho con el producto y desea cambiarlo. El producto debe estar todavía en el paquete original y sin uso.
4. Al cliente se le cobró más de una vez en su tarjeta de crédito el importe de la compra.

Bajo estas condiciones, las solicitudes de Shara y Chris no son elegibles para un reembolso ya que sacaron el producto de su empaque original e incluso la usaron por un tiempo. Las condiciones establecen claramente que los productos deben estar en su embalaje original y no deben tener uso. En las restantes solicitudes aplica un reembolso.

Ahora que tenemos las reglas, vamos a diseñar un prompt para indicarle al modelo que procese las solicitudes de reembolso. La idea es seguir un proceso de razonamiento paso a paso y llegar a una conclusión basada en el conjunto de reglas en lugar de simplemente adivinar. Siéntete libre de probar este prompt e incluso modificarlo para seguir un proceso de razonamiento diferente. Ten en cuenta que este es un buen ejemplo también para proporcionar ejemplos reales como parte del prompt (few-shot-prompting) y aprovechar de las capacidades de aprendizaje del modelo.

**Prompt 💬**

In [21]:
prompt = f"""
You will be provided with a list of user complaints delimited by <>, who want a refund of their purchase for different reasons. 
Your task is to determine if corresponds or not, to give the customer a refund based on the company policy.

Refunds policies

Only apply a refund in the following scenarios:
- The customer received the product with damages or in poor conditions
- The customer received the wrong size or other product. The product must be still in the original package without usage.
- The customer is not satisfied with the product and wishes to change it. The product must be still in the original package without usage.
- The customer was charged more than once in his credit card.

To solve the problem do the following:

1 - Analyze whether the purchase applies the refunds policy based on the reasons given by the customer.
2 - Categorize the case in: "apply" or "not apply" based on your previous judgment.
3 - Provide a concise and personalized response with the reasons for your previous solution.
5 - Output a json list object that contains the 
  following keys for each case: category, answer.

Tickets: <{refunds}>
"""

Como podemos ver en la respuesta, GPT-4 resuelve correctamente cada caso, aportando para cada uno una justificación acorde, señalando los motivos por los que NO aplica la devolución. Seguramente sea necesario probar en un conjunto de datos más grande y representativo antes de apresurarse a concluir que LLM puede manejar perfectamente esta tarea. Sin embargo, los resultados son muy alentadores.

**Response 🤖** &nbsp;&nbsp;<span class="label success">GPT-4</span>

```python
[
  {
    "category": "apply",
    "answer": "Dear Jack, we apologize for the inconvenience caused. Since you received a product with a different material than described and have already returned it, you are eligible for a refund. We will process your refund as soon as possible."
  },
  {
    "category": "apply",
    "answer": "Dear Tom, we're sorry to hear that the product arrived damaged and dirty. As per our refund policy, you are eligible for a replacement. Please return the damaged shirt, and we will send you a new one."
  },
  {
    "category": "apply",
    "answer": "Dear Bob, we apologize for the sizing issue you experienced. As you haven't used the jacket and it's still in the original package, you can return it. We will process your refund or exchange it for the correct size upon receiving the returned item."
  },
  {
    "category": "not apply",
    "answer": "Dear Shara, we appreciate your feedback on the sizing chart. However, since you have already used the jacket for a week, we cannot process an exchange or refund. We recommend referring to the sizing chart and contacting us before making future purchases to ensure a better fit."
  },
  {
    "category": "apply",
    "answer": "Dear Sasha, we apologize for the double charge on your credit card. We will refund the second charge immediately. Thank you for bringing this to our attention."
  },
  {
    "category": "not apply",
    "answer": "Dear Chris, we're sorry to hear that the backpack didn't meet your expectations. However, as the item has been used for two weeks, our refund policy does not apply in this case. We encourage you to read our refund policies carefully before making future purchases."
  }
]
```

In [22]:
# Now is your turn!
response = get_completion(prompt)
print(response)

[
  {
    "category": "not apply",
    "answer": "Based on the company's refund policy, a refund does not apply in this case. The reason for this is that the customer's complaint is about the product not meeting their personal preference in terms of material composition, which does not fall under the eligible reasons for a refund."
  },
  {
    "category": "apply",
    "answer": "Based on the company's refund policy, a refund applies in this case. The customer received the product with damages and in poor conditions, which is one of the eligible reasons for a refund. They can return the product and request a new unit."
  },
  {
    "category": "apply",
    "answer": "Based on the company's refund policy, a refund applies in this case. The customer received the wrong size and the product is still in the original package without usage, which is one of the eligible reasons for a refund. They can return the product and request the correct size."
  },
  {
    "category": "apply",
    "answe

## Tercer Principio: Balancear especificidad y creatividad
<a class="anchor" id="allow-creativity"></a>

🔝 [Ir al Índice](#index)

## Consejos útiles
<a class="anchor" id="tips-and-tricks"></a>

🔝 [Ir al Índice](#index)

The principles can help you as a general guide to writing clear and effective prompts. In addition, the following tips & tricks can help you optimize your prompts even more.

Los 3 principios van a ayudarte como una guía general para escribir instrucciones claras y efectivas. Adicionalmente, los siguientes consejos y trucos pueden resultarte de gran ayuda también:

**1- Iterar**

No existe tal cosa como un prompt perfecto, así que ve de una a probar el modelo con un prompt simple e imperfecto y luego has cambios y repite la ejecución: repetir, repetir, repetir. Puedes seguir un ciclo de cuatro pasos: (i) escribir tu prompt, (ii) ejecutarlo, (iii) analizar el/los error/es, (iv) encontrar mejoras y repetir el ciclo.


**2- Ser explícito y utilizar lenguaje objetivo**

Las palabras que no tienen un significado claro o cuyo significado puede ser subjetivo pueden llevar al modelo a confundirse. Por ejemplo, como parte de una instrucción, solicitar volver a escribir un texto de usuario para que se vea mejor. ¿Qué significa verse mejor? En su lugar, es preferible indicar de forma explícita cómo queremos mejorar el texto: limitar la extensión total a 50 palabras, corregir errores gramaticales, evitar lenguaje personal, etc.


**3- Enumerar las instrucciones**

En lugar de escribir un párrafo largo con instrucciones complejas, divídalo en instrucciones simples y enumerarlas.


**4- Evitar afirmaciones negadas**

Trate de evitar sentencias de negación, a veces puede llevar el modelo a confusiones. En su lugar, convertirlas en sentencias afirmativas. Por ejemplo, en lugar de escribir _“**no** utilice expresiones de lenguaje misógino o sexista”_, escribir _“**usar** lenguaje inclusivo y respetuoso”_.


**5- Reemplazar palabras**

Algunas palabras pueden ser más efectivas que otras, en términos de contextualizar y gyuar al modelo en la dirección correcta. Si tu prompt no esta dando los resultados esperados, tratá de reemplazar palabras importantes por sinónimos, siempre optando en favor de palabras más simples y frecuentes.

**6- Prompt Perfect**

Siempre podes escribir buenos prompts desde cero siguiendo los principios y buenas prácticas o también podes basarte en ejemplos de prompts ya hechos por alguien más. Por ejemplo [PromptPerfect](https://promptperfect.jina.ai/) ofrece una basta colección de Prompts ya diseñados por alguien más, al igual que una herramienta potenciada por AI que te permite tomar tu prompt "borrador" y optimizarlo para diferentes LLMs agregando instrucciones más claras y contexto. Algunas de las LLM soportadas son GPT-3, GPT-4, Midjourney, entre otras.


Si hablamos de templates de prompts, también tenemos que mencionar a los siguientes repositorios:


➡️ [500+ ChatGPT Prompt Templates](https://ignacio-velasquez.notion.site/500-ChatGPT-Prompt-Templates-d9541e901b2b4e8f800e819bdc0256da)

➡️ [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts/blob/main/prompts.csv)

Como consejo final, la vida no es solo sobre prompting! A que me refiero, es tan importante conocer y entender el modelo que estamos utlizando, como funciona, si acepta parámetros extra como parte del prompt (esto es muy comun en modeos `text-to-image` como Midjourney), como conocer las técnicas y principios detraś de un buen prompt. Por ejemplo, podemos ajustar el nivel de "creatividad" de un modelo como `ChatGPT` simplemente variando los parámetros `temperature` y `top_p`. ¿Lo sabías?

En conclusión, no te olvides nunca de leer la documentación!!

## Referencias
<a class="anchor" id="references"></a>

* [ChatGPT Prompt Engineering for Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)

* [The Full Stack LLM Bootcamp](https://fullstackdeeplearning.com/llm-bootcamp/)

* [OpenAI Playground](https://platform.openai.com/playground)

* [OpenAI Cookbook](https://github.com/openai/openai-cookbook)

* [How to generate text: using different decoding methods for language generation with Transformers](https://huggingface.co/blog/how-to-generate)

* [Holtzman, A., Buys, J., Du, L., Forbes, M., & Choi, Y. (2019). The curious case of neural text degeneration. arXiv preprint arXiv:1904.09751.](https://arxiv.org/abs/1904.09751)